In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import glob # lista di files in formato *.estensione
import matplotlib.pyplot as plt
from PIL import Image # import immagine e trasforma in black-white

In [ ]:
PATH_DATA = "./hasy-data/"
FILES = glob.glob(PATH_DATA + "*.png") # lista di tutte le immagini nella cartella

In [ ]:
# Importa label delle immagini
labels = pd.read_csv("./hasy-data-labels.csv")
y = labels.latex
y.to_pickle("./y.pkl")

In [ ]:
cols = ["pixel_" + str(i) + "_" + str(j) for i in range(32) for j in range(32)] # colonne: pixel_x_y

dati = np.empty((len(FILES), 1024), dtype = int)
for i in range(len(FILES)):
    dati[i] = np.array(Image.open(FILES[i]).convert("1")).reshape(1,1024)
    if not i%1000:
        print(i)
dati = pd.DataFrame(dati, columns = cols)

In [ ]:
# Trasformo dati in 1 == nero, 0 == bianco
dati = 1 - dati

In [ ]:
dati.to_pickle("./dati.pkl")